In [1]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Hyperparameters
LR = 1e-3        # learning rate for Adam
EPOCHS = 5000    # number of training epochs (increase for better results)
N_HIDDEN = 50    # neurons per hidden layer
N_LAYERS = 4     # number of hidden layers
N_PDE = 2000     # number of interior points for PDE
N_BC = 200       # number of points per boundary segment

# Material and problem parameters
lambda_ = 1.0
mu = 0.5
Q = 4.0

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

In [ ]:
'''
As PINN DL engineer, what are the points to adjust in this network and training:

1) N_LAYERS, N_HIDDEN → Network capacity adjustment
2) N_PDE, N_BC → collocation / boundary sampling density
3) loss = w_pde * loss_pde + w_bc * loss_bc: Weight tuninig
4) activation(tanh → swish, sine, etc.) function switch
5) initialization method change, optimizer/learning rate scheduel adjustment
'''


# ============================================================
# 1. Neural network model: PINN for 2D elasticity
#    Input:  (x, y)
#    Output: (u_x, u_y)  -> displacement field components
# ============================================================

class PINN(nn.Module):
    def __init__(self, layers, neurons):
        """
        layers  : number of hidden layers (depth)
        neurons : number of neurons per hidden layer (width)

        Network architecture (schematic):
          Input (x, y) ∈ R^2
            → Linear(2 -> neurons) + Tanh
            → [layers-1] × [Linear(neurons -> neurons) + Tanh]
            → Linear(neurons -> 2)  (outputs: [u_x, u_y])
        """
        super().__init__()

        # First fully-connected layer: maps 2D input (x,y) to 'neurons'-dim hidden features.
        self.input_layer = nn.Linear(2, neurons)

        # List of hidden layers, each Linear(neurons -> neurons).
        # nn.ModuleList is used so that PyTorch can register all layers'
        # parameters correctly (so they are updated during training).
        self.hidden = nn.ModuleList([
            nn.Linear(neurons, neurons) for _ in range(layers - 1)
        ])

        # Final output layer: maps hidden features → 2 outputs.
        # Here, the two outputs correspond to displacement components:
        # u_x(x,y) and u_y(x,y).
        self.output_layer = nn.Linear(neurons, 2)

        # Activation function used after each Linear layer (except the last).
        # Tanh is commonly chosen in PINNs because it is smooth and
        # infinitely differentiable (good for higher-order PDE derivatives).
        self.activation = nn.Tanh()
        
        # --------------------------------------------------------
        # Weight initialization: Xavier normal, adapted for tanh.
        # This helps keep signal/gradient magnitudes stable
        # across layers, improving training stability.
        # --------------------------------------------------------
        for m in self.hidden:
            nn.init.xavier_normal_(
                m.weight,
                gain=nn.init.calculate_gain('tanh')
            )
        nn.init.xavier_normal_(
            self.input_layer.weight,
            gain=nn.init.calculate_gain('tanh')
        )
        # Output layer often does not use an activation; we can use
        # Xavier normal with default gain (=1).
        nn.init.xavier_normal_(self.output_layer.weight)
        
    def forward(self, x):
        """
        Forward pass of the network.

        Input:
          x : tensor of shape (N, 2) where each row is (x_i, y_i).

        Steps:
          1) Apply input_layer + tanh.
          2) Pass through each hidden layer with tanh.
          3) Apply final output_layer (no activation).
        
        Output:
          tensor of shape (N, 2), each row [u_x(x_i, y_i), u_y(x_i, y_i)].
        """
        # First layer + activation
        z = self.activation(self.input_layer(x))

        # Propagate through all hidden layers, applying tanh each time
        for m in self.hidden:
            z = self.activation(m(z))

        # Final linear layer outputs 2D displacement vector at each point
        return self.output_layer(z)


# Instantiate the model using chosen hyperparameters:
#   N_LAYERS : number of hidden layers
#   N_HIDDEN : number of neurons per hidden layer
#   device   : 'cpu' or 'cuda'
model = PINN(N_LAYERS, N_HIDDEN).to(device)



# ============================================================
# 2. Boundary force (body force / source terms) for elasticity PDE
#    These are the right-hand side terms f_x(x,y), f_y(x,y)
#    in the equilibrium equations:
#
#   ∂σ_xx/∂x + ∂σ_xy/∂y + f_x = 0
#   ∂σ_xy/∂x + ∂σ_yy/∂y + f_y = 0
#   
#   lambda_, mu : Lamé parameters (material constants)
#   Q          : some loading parameter
# ============================================================

# Body forces
def f_x_fun(x, y):
    """
    Compute x-component of the body force f_x(x,y).

    term1, term2 correspond to contributions from lambda_ and mu
    in a manufactured (constructed) forcing function that is
    consistent with a chosen analytical displacement field.
    """
    term1 = lambda_ * (
        4.0 * np.pi**2 * torch.cos(2.0 * np.pi * x) * torch.sin(np.pi * y)
        - np.pi * torch.cos(np.pi * x) * Q * (y**3)
    )
    term2 = mu * (
        9.0 * np.pi**2 * torch.cos(2.0 * np.pi * x) * torch.sin(np.pi * y)
        - np.pi * torch.cos(np.pi * x) * Q * (y**3)
    )
    return term1 + term2


def f_y_fun(x, y):
    """
    Compute y-component of the body force f_y(x,y).

    Again, lambda_ and mu terms are grouped as term1 and term2, and the
    structure is chosen so that the PDE has a known analytical solution
    (manufactured solution technique).
    """
    term1 = lambda_ * (
        -3.0 * torch.sin(np.pi * x) * Q * (y**2)
        + 2.0 * np.pi**2 * torch.sin(2.0 * np.pi * x) * torch.cos(np.pi * y)
    )
    term2 = mu * (
        -6.0 * torch.sin(np.pi * x) * Q * (y**2)
        + 2.0 * np.pi**2 * torch.sin(2.0 * np.pi * x) * torch.cos(np.pi * y)
        + (np.pi**2) * torch.sin(np.pi * x) * Q * (y**4) / 4.0
    )
    return term1 + term2



# ============================================================
# 3. Sampling training points: interior (PDE) + boundary (BC)
# ============================================================

## Interior PDE collocation points
# Randomly sample N_PDE points in the unit square [0,1]^2.
# These are the points where we enforce the PDE residual (equilibrium equations).
xy_pde = np.random.rand(N_PDE, 2)  # shape: (N_PDE, 2) with entries in [0,1]
xy_pde_torch = torch.tensor(xy_pde, dtype=torch.float32).to(device)


# ------------------------------------------------------------
# Boundary points: each side of the unit square
#   y = 0, y = 1, x = 0, x = 1
# ------------------------------------------------------------

# y = 0 boundary: x ∈ [0,1]
bc_y0 = np.linspace(0, 1, N_BC)                   # param along x
xy_bc_y0 = np.column_stack((bc_y0, np.zeros_like(bc_y0)))  # (x, 0)

# y = 1 boundary: x ∈ [0,1]
bc_y1 = np.linspace(0, 1, N_BC)
xy_bc_y1 = np.column_stack((bc_y1, np.ones_like(bc_y1)))   # (x, 1)

# x = 0 boundary: y ∈ [0,1]
bc_x0 = np.linspace(0, 1, N_BC)
xy_bc_x0 = np.column_stack((np.zeros_like(bc_x0), bc_x0))  # (0, y)

# x = 1 boundary: y ∈ [0,1]
bc_x1 = np.linspace(0, 1, N_BC)
xy_bc_x1 = np.column_stack((np.ones_like(bc_x1), bc_x1))   # (1, y)

# Convert all boundary point arrays to PyTorch tensors on the chosen device
xy_bc_y0_torch = torch.tensor(xy_bc_y0, dtype=torch.float32).to(device)
xy_bc_y1_torch = torch.tensor(xy_bc_y1, dtype=torch.float32).to(device)
xy_bc_x0_torch = torch.tensor(xy_bc_x0, dtype=torch.float32).to(device)
xy_bc_x1_torch = torch.tensor(xy_bc_x1, dtype=torch.float32).to(device)





# ============================================================
# 4. PDE residual loss L_PDE
#    Implements the elasticity equilibrium equations:
#
#   ∂σ_xx/∂x + ∂σ_xy/∂y + f_x = 0
#   ∂σ_xy/∂x + ∂σ_yy/∂y + f_y = 0
#
#   where σ_xx, σ_yy, σ_xy are stresses derived from displacement u.
# ============================================================

def loss_pde(xy):
    """
    Compute the mean-squared PDE residual over a batch of interior points.

    Input:
      xy : tensor of shape (N, 2) with coordinates (x_i, y_i).

    Steps:
      1) Enable autograd on xy to differentiate displacement w.r.t x,y.
      2) Compute displacement u(x,y) = [u_x, u_y] from the PINN.
      3) Compute spatial derivatives u_{x,x}, u_{x,y}, u_{y,x}, u_{y,y}.
      4) Build stress components σ_xx, σ_yy, σ_xy via linear elasticity.
      5) Take divergence of stress + body force:
         r1 = ∂σ_xx/∂x + ∂σ_xy/∂y + f_x
         r2 = ∂σ_xy/∂x + ∂σ_yy/∂y + f_y
      6) Return mean squared residual: E[r1^2 + r2^2].
    """
    # Clone/detach so we don't accidentally backprop through earlier graph;
    # requires_grad_(True) so that autograd tracks derivatives w.r.t x,y.
    xy_ = xy.clone().detach().requires_grad_(True)
    x = xy_[:, 0:1]  # shape: (N,1)
    y = xy_[:, 1:2]  # shape: (N,1)

    # Forward pass: displacement field u(x,y) from PINN
    # We concatenate x and y along feature dimension to get shape (N,2).
    u = model(torch.cat((x, y), dim=1))  # shape: (N,2)
    u_x = u[:, 0:1]  # displacement component in x-direction
    u_y = u[:, 1:2]  # displacement component in y-direction

    # ----------------------
    # Compute first derivatives of displacement
    # ----------------------

    # ∂u_x/∂x ~ ε_xx (strain component)
    grad_u_x_x = torch.autograd.grad(
        u_x, x,
        torch.ones_like(u_x),
        create_graph=True)[0]

    # ∂u_x/∂y ~ ε_xy (strain component)
    grad_u_x_y = torch.autograd.grad(
        u_x, y,
        torch.ones_like(u_x),
        create_graph=True)[0]

    # ∂u_y/∂x ~ ε_yx (strain component)
    grad_u_y_x = torch.autograd.grad(
        u_y, x,
        torch.ones_like(u_y),
        create_graph=True)[0]

    # ∂u_y/∂y ~ ε_yy (strain component)
    grad_u_y_y = torch.autograd.grad(
        u_y, y,
        torch.ones_like(u_y),
        create_graph=True)[0]

    
    # ----------------------
    # Constitutive relations
    # Build stress tensor components using linear elasticity (plane strain)
    # For isotropic linear elasticity:
    #   1) σ_xx = (λ + 2μ) ∂u_x/∂x + λ ∂u_y/∂y
    #   2) σ_yy = (λ + 2μ) ∂u_y/∂y + λ ∂u_x/∂x
    #   3) σ_xy = μ (∂u_x/∂y + ∂u_y/∂x)
    #   μ ~ shear modulus
    # ----------------------
    sigma_xx = (lambda_ + 2.0 * mu) * grad_u_x_x + lambda_ * grad_u_y_y   # 1)
    sigma_yy = (lambda_ + 2.0 * mu) * grad_u_y_y + lambda_ * grad_u_x_x   # 2)
    sigma_xy = mu * (grad_u_x_y + grad_u_y_x)                             # 3)

    # ----------------------
    # Derivatives of stresses: ∂σ_xx/∂x, ∂σ_xy/∂y, ∂σ_xy/∂x, ∂σ_yy/∂y
    # ----------------------
    sigma_xx_x = torch.autograd.grad(
        sigma_xx, x,
        torch.ones_like(sigma_xx),
        create_graph=True)[0]
    
    sigma_xy_y = torch.autograd.grad(
        sigma_xy, y,
        torch.ones_like(sigma_xy),
        create_graph=True)[0]
    
    sigma_xy_x = torch.autograd.grad(
        sigma_xy, x,
        torch.ones_like(sigma_xy),
        create_graph=True)[0]
    
    sigma_yy_y = torch.autograd.grad(
        sigma_yy, y,
        torch.ones_like(sigma_yy),
        create_graph=True)[0]

    # Body force components at (x,y)
    f_x = f_x_fun(x, y)
    f_y = f_y_fun(x, y)

    # ----------------------
    # PDE residuals (equilibrium in x and y directions)
    # ----------------------
    r1 = sigma_xx_x + sigma_xy_y + f_x  # should be ~ 0
    r2 = sigma_xy_x + sigma_yy_y + f_y  # should be ~ 0

    # Mean squared residual over all interior points
    return torch.mean(r1**2 + r2**2)



# ============================================================
# 5. Boundary condition loss L_BC
#    Enforces displacement and traction conditions on:
#      - y = 0
#      - y = 1
#      - x = 0
#      - x = 1
# ============================================================

def loss_bc():
    """
    Compute boundary condition loss over all four sides of the domain.

    Boundary conditions (schematic):

      y = 0:
        u_x = 0, u_y = 0      (clamped or fixed boundary)

      y = 1:
        u_x = 0, σ_yy = (λ + 2μ) Q sin(π x)   (prescribed normal traction in y)

      x = 0:
        u_y = 0, σ_xx = 0     (mix of displacement & traction conditions)

      x = 1:
        u_y = 0, σ_xx = 0

    Each condition adds a mean-squared penalty term to loss_val.
    """
    loss_val = 0.0
    
    # --------------------------------------------------------
    # Boundary: y = 0  => u_x = 0, u_y = 0
    # --------------------------------------------------------
    # Extract x,y on this boundary and enable gradients for computing stresses if needed.
    x0 = xy_bc_y0_torch[:, 0:1].clone().detach().requires_grad_(True)  # x ∈ [0,1]
    y0 = xy_bc_y0_torch[:, 1:2].clone().detach().requires_grad_(True)  # y = 0

    # Displacement at (x, 0)
    u_bc_y0 = model(torch.cat((x0, y0), dim=1))  # shape (N_BC, 2)

    # Enforce u_x(x,0) = 0 and u_y(x,0) = 0
    loss_val += torch.mean(u_bc_y0[:, 0:1]**2)  # penalty for u_x
    loss_val += torch.mean(u_bc_y0[:, 1:2]**2)  # penalty for u_y
    
    # --------------------------------------------------------
    # Boundary: y = 1  => u_x = 0,  σ_yy = (λ+2μ) Q sin(π x)
    # --------------------------------------------------------
    x1 = xy_bc_y1_torch[:, 0:1].clone().detach().requires_grad_(True)  # x ∈ [0,1]
    y1 = xy_bc_y1_torch[:, 1:2].clone().detach().requires_grad_(True)  # y = 1

    u_bc_y1 = model(torch.cat((x1, y1), dim=1))  # displacement at (x,1)

    # Enforce u_x(x,1) = 0
    loss_val += torch.mean(u_bc_y1[:, 0:1]**2)

    # For σ_yy, we need ∂u_y/∂y and ∂u_x/∂x at this boundary:
    u_y_bc_y1 = u_bc_y1[:, 1:2]
    u_y_bc_y1_y = torch.autograd.grad(
        u_y_bc_y1, y1,
        torch.ones_like(u_y_bc_y1),
        create_graph=True
    )[0]

    u_x_bc_y1 = u_bc_y1[:, 0:1]
    u_x_bc_y1_x = torch.autograd.grad(
        u_x_bc_y1, x1,
        torch.ones_like(u_x_bc_y1),
        create_graph=True
    )[0]

    # σ_yy = (λ + 2μ) ∂u_y/∂y + λ ∂u_x/∂x
    sigma_yy_y1 = (lambda_ + 2.0 * mu) * u_y_bc_y1_y + lambda_ * u_x_bc_y1_x

    # Target traction σ_yy on y=1:
    sigma_yy_target = (lambda_ + 2.0 * mu) * Q * torch.sin(np.pi * x1)

    # Penalize deviation from prescribed traction
    loss_val += torch.mean((sigma_yy_y1 - sigma_yy_target)**2)
    
    # --------------------------------------------------------
    # Boundary: x = 0  => u_y = 0,  σ_xx = 0
    # --------------------------------------------------------
    x0_ = xy_bc_x0_torch[:, 0:1].clone().detach().requires_grad_(True)  # x = 0
    y0_ = xy_bc_x0_torch[:, 1:2].clone().detach().requires_grad_(True)  # y ∈ [0,1]

    u_bc_x0 = model(torch.cat((x0_, y0_), dim=1))  # displacement along x=0

    # Enforce u_y(0,y) = 0
    loss_val += torch.mean(u_bc_x0[:, 1:2]**2)

    # For σ_xx, need ∂u_x/∂x and ∂u_y/∂y
    u_x_bc_x0 = u_bc_x0[:, 0:1]
    u_x_bc_x0_x = torch.autograd.grad(
        u_x_bc_x0, x0_,
        torch.ones_like(u_x_bc_x0),
        create_graph=True
    )[0]

    u_y_bc_x0 = u_bc_x0[:, 1:2]
    u_y_bc_x0_y = torch.autograd.grad(
        u_y_bc_x0, y0_,
        torch.ones_like(u_y_bc_x0),
        create_graph=True
    )[0]

    # σ_xx = (λ + 2μ) ∂u_x/∂x + λ ∂u_y/∂y
    sigma_xx_x0 = (lambda_ + 2.0 * mu) * u_x_bc_x0_x + lambda_ * u_y_bc_x0_y

    # Enforce σ_xx(0,y) = 0
    loss_val += torch.mean(sigma_xx_x0**2)
    
    # --------------------------------------------------------
    # Boundary: x = 1  => u_y = 0,  σ_xx = 0
    # --------------------------------------------------------
    x1_ = xy_bc_x1_torch[:, 0:1].clone().detach().requires_grad_(True)  # x = 1
    y1_ = xy_bc_x1_torch[:, 1:2].clone().detach().requires_grad_(True)  # y ∈ [0,1]

    u_bc_x1 = model(torch.cat((x1_, y1_), dim=1))  # displacement along x=1

    # Enforce u_y(1,y) = 0
    loss_val += torch.mean(u_bc_x1[:, 1:2]**2)

    # For σ_xx on x=1:
    u_x_bc_x1 = u_bc_x1[:, 0:1]
    u_x_bc_x1_x = torch.autograd.grad(
        u_x_bc_x1, x1_,
        torch.ones_like(u_x_bc_x1),
        create_graph=True
    )[0]

    u_y_bc_x1 = u_bc_x1[:, 1:2]
    u_y_bc_x1_y = torch.autograd.grad(
        u_y_bc_x1, y1_,
        torch.ones_like(u_y_bc_x1),
        create_graph=True
    )[0]

    sigma_xx_x1 = (lambda_ + 2.0 * mu) * u_x_bc_x1_x + lambda_ * u_y_bc_x1_y

    # Enforce σ_xx(1,y) = 0
    loss_val += torch.mean(sigma_xx_x1**2)

    return loss_val




In [ ]:
# ---------------------------------------------------------
# Optimizer setup
#   - Adam optimizer updates all model parameters (weights + biases)
#   - LR is a user-defined learning rate constant
# ---------------------------------------------------------
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# ---------------------------------------------------------
# Main training loop
#   - EPOCHS: total number of optimization steps
#   - At each epoch:
#       1) Reset gradients
#       2) Compute PDE (interior) loss
#       3) Compute boundary-condition loss
#       4) Combine into total loss
#       5) Backpropagate (compute gradients)
#       6) Update parameters with optimizer
# ---------------------------------------------------------
for ep in range(EPOCHS):
    # Clear any gradients from the previous step
    optimizer.zero_grad()

    # PDE (interior) loss:
    #   - xy_pde_torch contains interior collocation points (x, y)
    #   - loss_pde measures violation of the PDE equilibrium at these points
    loss_interior = loss_pde(xy_pde_torch)

    # Boundary-condition loss:
    #   - loss_bc enforces displacement and/or traction conditions
    #     on the domain boundaries (e.g., x=0,1 or y=0,1)
    loss_bound = loss_bc()

    # Total loss = PDE residual loss + boundary loss
    #   - As written, both contributions are equally weighted
    #   - In practice, you can introduce weighting factors if needed
    loss_total = loss_interior + loss_bound

    # Backpropagation:
    #   - Compute gradients of loss_total with respect to all trainable parameters
    loss_total.backward()

    # Optimizer step:
    #   - Apply gradient-based updates to model parameters
    optimizer.step()

    # Print losses periodically for monitoring convergence
    if ep % 500 == 0:
        print(
            f"Epoch {ep:5d} | PDE Loss: {loss_interior.item():.4e} | "
            f"BC Loss: {loss_bound.item():.4e} | Total: {loss_total.item():.4e}"
        )

print("Training finished.")


# ---------------------------------------------------------
# Evaluation and visualization
#   - Assume XX, YY define a regular 2D grid over the domain
#   - Evaluate the trained PINN at each grid point to get u_x, u_y
#   - Compare with exact solution fields u_x_ex, u_y_ex
# ---------------------------------------------------------

# Stack grid coordinates into a list of (x, y) points:
#   - XX.ravel() and YY.ravel() flatten the 2D grids into 1D vectors
#   - np.column_stack builds an array of shape (N_points, 2)
XY_plot = np.column_stack((XX.ravel(), YY.ravel()))

# Convert the coordinate array to a PyTorch tensor and move it to the chosen device
XY_torch = torch.tensor(XY_plot, dtype=torch.float32).to(device)

# Disable gradient tracking for evaluation to save memory and computation
with torch.no_grad():
    # Evaluate the model at all grid points
    # pred has shape (N_points, 2), where:
    #   pred[:, 0] ≈ u_x(x, y)
    #   pred[:, 1] ≈ u_y(x, y)
    pred = model(XY_torch).cpu().numpy()

# Reshape PINN predictions back to 2D fields matching XX, YY shapes:
#   - u_x_pred and u_y_pred become (ny, nx) arrays
u_x_pred = pred[:, 0].reshape((ny, nx))
u_y_pred = pred[:, 1].reshape((ny, nx))

# Compute exact (analytic) displacement fields on the same grid
#   - exact_u_x(XX, YY) and exact_u_y(XX, YY) are user-defined functions
u_x_ex = exact_u_x(XX, YY)
u_y_ex = exact_u_y(XX, YY)



# ---------------------------------------------------------
# Plot 1: Compare u_x (PINN vs exact)
# ---------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left subplot: PINN-predicted u_x field
cont1 = axs[0].contourf(XX, YY, u_x_pred, levels=30)
fig.colorbar(cont1, ax=axs[0])
axs[0].set_title("PINN predicted $u_x$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right subplot: exact u_x field
cont2 = axs[1].contourf(XX, YY, u_x_ex, levels=30)
fig.colorbar(cont2, ax=axs[1])
axs[1].set_title("Exact $u_x$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()



# ---------------------------------------------------------
# Plot 2: Compare u_y (PINN vs exact)
# ---------------------------------------------------------
fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Left subplot: PINN-predicted u_y field
cont3 = axs[0].contourf(XX, YY, u_y_pred, levels=30)
fig.colorbar(cont3, ax=axs[0])
axs[0].set_title("PINN predicted $u_y$")
axs[0].set_xlabel("x")
axs[0].set_ylabel("y")

# Right subplot: exact u_y field
cont4 = axs[1].contourf(XX, YY, u_y_ex, levels=30)
fig.colorbar(cont4, ax=axs[1])
axs[1].set_title("Exact $u_y$")
axs[1].set_xlabel("x")
axs[1].set_ylabel("y")

plt.tight_layout()
plt.show()
